<a href="https://colab.research.google.com/github/claudiobarril/pln1_17co2024/blob/main/Desafio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Desafío 2: Custom embedddings con Gensim

### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizarán los cuentos de "Las mil y una noches" para generar los embeddings, es decir, que los vectores tendrán la forma en función de como se hayan utilizado las palabras en dichas historias.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import multiprocessing
from gensim.models import Word2Vec

import requests
import fitz
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

from scipy.spatial.distance import cosine

import fitz
import re
import pickle

from sklearn.manifold import TSNE
import plotly.express as px

Convertir PDF descargado de https://www.textos.info/anonimo/las-mil-y-una-noches a TXT

In [ ]:
def pdf_to_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])  # Extraer texto de cada página
    return text

# Cargar y guardar el texto
pdf_text = pdf_to_text("desafio_2/Anonimo - Las Mil y Una Noches.pdf")
with open("desafio_2/las-mil-y-una-noches.txt", "w", encoding="utf-8") as f:
    f.write(pdf_text)

#### Construcción del corpus

Vamos a probar dos estratégias distintas. Crear un documento por cada línea del libro, y un documento por cada página.
Sería interesante probar un documento por cuento, pero la separación del libro en cuentos no es trivial, y debería utilizarse una lista de todos los cuentos para dicha tarea. Se abordó esa idea sin éxito en un tiempo prudente, por lo que fue descartada.

Como lado positivo, vamos a tener documentos cortos y contexto local, aunque como contra punto, puede que dicho contexto local se pierda en varias situaciones, dado que cortamos indiscriminadamente cuentos por la mitad, ya sea por línea o por página.

##### Un documento por página

In [ ]:
# Leer el archivo .txt completo
with open("desafio_2/las-mil-y-una-noches.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Dividir por páginas usando saltos dobles de línea
pages = text.split("\n\n")

# Filtrar líneas que son solo números (números de página)
clean_pages = [re.sub(r"^\d+$", "", page, flags=re.MULTILINE).strip() for page in pages]

# Eliminar entradas vacías después de la limpieza
clean_pages = [page for page in clean_pages if page]

# Quitar las primeras 6 páginas (introducción)
clean_pages = clean_pages[6:]

# Crear un DataFrame con cada página limpia como un documento
df_by_pages = pd.DataFrame(clean_pages, columns=["text"])

# Mostrar algunas páginas procesadas
for i, page in enumerate(df_by_pages["text"].head(7)):
    print(f"\n--- Página {i+1} ---\n")
    print(page[:500])  # Mostrar los primeros 500 caracteres de cada fragmento

print("\nCantidad de documentos:", df_by_pages.shape[0])


--- Página 1 ---

Historia del rey Schahriar y su hermano el rey 
Schahzaman
Cuéntase —pero Alah es más sabio, más prudente más poderoso y más 
benéfico— que en lo que transcurrió en la antigüedad del tiempo y en lo 
pasado de la edad, hubo un rey entre los reyes de Sassan, en las islas de 
la India y de la China. Era dueño de ejércitos y señor de auxiliares, de 
servidores y de un séquito numeroso. Tenía dos hijos, y ambos eran 
heroicos jinetes, pero el mayor valía más aún que el menor. El mayor reinó 
en los p

--- Página 2 ---

hermano?» Desenvainó inmediatamente su alfanje, y acometiendo a 
ambos, los dejó muertos sobre los tapices del lecho. Volvió a salir sin 
perder una hora ni un instante, y ordenó la marcha de la comitiva. Y viajó 
de noche, hasta avistar la ciudad de su hermano.
Entonces éste se alegró de su proximidad, salió a su encuentro, y al 
recibirlo, le deseó la paz. Se regocijó hasta los mayores límites del 
contento, mandó adornar en honor suyo la ciudad, y se pus

In [ ]:
sentence_tokens_by_pages = [text_to_word_sequence(page) for page in clean_pages]

### 2 - Crear los vectores (word2vec)

In [ ]:
# Creamos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model_by_pages = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,# dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=4,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

# Obtener el vocabulario con los tokens
w2v_model_by_pages.build_vocab(sentence_tokens_by_pages)

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model_by_pages.corpus_count)

# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model_by_pages.wv.index_to_key))

Cantidad de docs en el corpus: 3626
Cantidad de words distintas en el corpus: 12882


##### Un documento por línea

In [ ]:
# Leer el archivo .txt línea por línea
with open("desafio_2/las-mil-y-una-noches.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Limpiar cada línea: quitar espacios extra y números de página
clean_lines = [re.sub(r"^\d+$", "", line.strip()) for line in lines]

# Eliminar líneas vacías después de la limpieza
clean_lines = [line for line in clean_lines if line]

# Quitar las primeras 140 líneas (introducción)
clean_lines = clean_lines[126:]

# Crear DataFrame con cada línea como un documento
df_by_lines = pd.DataFrame(clean_lines, columns=["text"])

# Mostrar algunas líneas procesadas
for i, line in enumerate(df_by_lines["text"].head(10)):
    print(f"Línea {i}: {line}")  # Mostrar la línea completa

print("\nCantidad de documentos:", df_by_lines.shape[0])

Línea 0: Historia del rey Schahriar y su hermano el rey
Línea 1: Schahzaman
Línea 2: Cuéntase —pero Alah es más sabio, más prudente más poderoso y más
Línea 3: benéfico— que en lo que transcurrió en la antigüedad del tiempo y en lo
Línea 4: pasado de la edad, hubo un rey entre los reyes de Sassan, en las islas de
Línea 5: la India y de la China. Era dueño de ejércitos y señor de auxiliares, de
Línea 6: servidores y de un séquito numeroso. Tenía dos hijos, y ambos eran
Línea 7: heroicos jinetes, pero el mayor valía más aún que el menor. El mayor reinó
Línea 8: en los países, gobernó con justicia entre los hombres y por eso le querían
Línea 9: los habitantes del país y del reino. Llamábase el rey Schahriar. Su

Cantidad de documentos: 105622


In [ ]:
# Tokenizar cada línea en palabras
sentence_tokens_by_lines = [text_to_word_sequence(line) for line in clean_lines]

# Definir y entrenar el modelo Word2Vec
w2v_model_by_lines = Word2Vec(
    min_count=5,    # Frecuencia mínima de palabra
    window=2,       # Contexto a considerar (palabras antes y después)
    vector_size=300,# Dimensión de los embeddings
    negative=20,    # Negative sampling
    workers=4,      # Número de núcleos para entrenamiento
    sg=1            # Skip-gram (1) en lugar de CBOW (0)
)

# Construir vocabulario
w2v_model_by_lines.build_vocab(sentence_tokens_by_lines)

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model_by_lines.corpus_count)

# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model_by_lines.wv.index_to_key))

Cantidad de docs en el corpus: 105622
Cantidad de words distintas en el corpus: 12882


### 3 - Entrenar embeddings

In [ ]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Entrenamos el modelo generador de vectores, por página
# Utilizamos nuestro callback
w2v_model_by_pages.train(sentence_tokens_by_pages,
    total_examples=w2v_model_by_pages.corpus_count,
    epochs=50,
    compute_loss = True,
    callbacks=[callback()]
)

Loss after epoch 0: 2336729.5
Loss after epoch 1: 1775423.0
Loss after epoch 2: 1626203.0


KeyboardInterrupt: 

In [ ]:
# Entrenamos el modelo generador de vectores, por línea
w2v_model_by_lines.train(sentence_tokens_by_lines,
    total_examples=w2v_model_by_lines.corpus_count,
    epochs=50,
    compute_loss = True,
    callbacks=[callback()]
)

In [ ]:
# Guardamos los modelos
with open('models/w2v_model_by_pages.pkl', 'wb') as file:
    pickle.dump(w2v_model_by_pages, file)

with open('models/w2v_model_by_lines.pkl', 'wb') as file:
    pickle.dump(w2v_model_by_lines, file)

### 4 - Ensayar

In [ ]:
# Cargamos los modelos desde archivo (no es necesario si los entrenamos en esta corrida)
with open('models/w2v_model_by_pages.pkl', 'rb') as file:
    w2v_model_by_pages = pickle.load(file)

with open('models/w2v_model_by_lines.pkl', 'rb') as file:
    w2v_model_by_lines = pickle.load(file)

In [ ]:
w2v_model_by_pages.wv.most_similar(positive=["lámpara"], topn=10)

In [ ]:
w2v_model_by_lines.wv.most_similar(positive=["lámpara"], topn=10)

In [ ]:
import pandas as pd

def compare_models(word, model_by_pages, model_by_lines, topn=10):
    # Obtener palabras más similares en ambos modelos
    similar_by_pages = model_by_pages.wv.most_similar(positive=[word], topn=topn)
    similar_by_lines = model_by_lines.wv.most_similar(positive=[word], topn=topn)

    # Convertir a listas formateadas
    words_pages = [f"{w[0]} ({w[1]:.2f})" for w in similar_by_pages]
    words_lines = [f"{w[0]} ({w[1]:.2f})" for w in similar_by_lines]

    # Asegurar que ambas listas tengan el mismo tamaño
    max_len = max(len(words_pages), len(words_lines))
    words_pages += [""] * (max_len - len(words_pages))
    words_lines += [""] * (max_len - len(words_lines))

    # Crear tabla formateada
    print("\n" + "=" * 50)
    print(f"Comparación para la palabra: {word}")
    print("=" * 50)
    print("   Model By Pages   |  Model By Lines ")
    print("-" * 50)

    for w1, w2 in zip(words_pages, words_lines):
        print(f" {w1.ljust(18)} | {w2}")

    print("-" * 50)  # Línea final separadora

# Lista de palabras a comparar
words = ["lámpara", "sultán", "genio", "alfombra", "califa", "anillo", "príncipe", "emir", "mezquita", "palacio", "seda", "puñal", "incienso", "destino"]

# Generar comparación para cada palabra
for word in words:
    compare_models(word, w2v_model_by_pages, w2v_model_by_lines)

Podemos observar muchas palabras con asociaciones semánticamente relacionadas. Por ejemplo:
* "lámpara" se asocia con "mágica", "ánfora" y "antorcha", lo cual encaja con la idea de una lámpara mágica en contextos como "Las Mil y Una Noches".
* "genio" tiene asociaciones con términos como "gigantesco" y "sucio", que podrían reflejar características típicas en descripciones de genios en la literatura.
* "anillo" tiene fuertes asociaciones con "talismánico", "engarce" y "brazalete", lo que sugiere que se vincula a conceptos de joyería y magia.

Ambos modelos parecen generar listas con relaciones similares, pero con algunas variaciones en las puntuaciones y las palabras asociadas. En algunos casos, el modelo por páginas tiende a producir términos más literarios o contextuales, mientras que el modelo por líneas parece generar términos más variados y específicos.

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model_by_pages.wv.most_similar(negative=["diedaa"])

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_palacio = w2v_model_by_pages.wv.get_vector("palacio")
print(vector_palacio)

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_by_pages.wv.most_similar(vector_palacio)

### 5 - Visualizar agrupación de vectores


In [ ]:
def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
vecs, labels = reduce_dimensions(w2v_model_by_pages)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS, 0], y=vecs[:MAX_WORDS, 1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D
vecs, labels = reduce_dimensions(w2v_model_by_pages, 3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS, 0], y=vecs[:MAX_WORDS, 1], z=vecs[:MAX_WORDS, 2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/

vectors = np.asarray(w2v_model_by_pages.wv.vectors)
labels = list(w2v_model_by_pages.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### 6 - Tests de analogías

In [ ]:
# Definir la función de analogía
def analogy(model, word_a, word_b, word_c, topn=10):
    """
    Encuentra la palabra D que complete la analogía: A es a B como C es a ?

    Parámetros:
    - model: modelo Word2Vec entrenado
    - word_a, word_b, word_c: palabras en la analogía
    - topn: cantidad de palabras más similares a mostrar

    Retorna:
    - Lista de palabras similares con sus puntuaciones
    """
    try:
        result = model.wv.most_similar(positive=[word_b, word_c], negative=[word_a], topn=topn)
        return result
    except KeyError as e:
        print(f"Error: {e}. Alguna palabra no está en el vocabulario.")
        return None

In [ ]:
def analogy_compare_models(model_by_pages, model_by_lines, word_a, word_b, word_c, topn=10):
    """
    Compara los resultados de la analogía en dos modelos de Word2Vec.

    Parámetros:
    - model_by_pages: modelo Word2Vec entrenado por páginas
    - model_by_lines: modelo Word2Vec entrenado por líneas
    - word_a, word_b, word_c: palabras en la analogía (A es a B como C es a ?)
    - topn: cantidad de palabras más similares a mostrar
    """

    # Obtener resultados de analogía para ambos modelos
    similar_by_pages = analogy(model_by_pages, word_a, word_b, word_c, topn)
    similar_by_lines = analogy(model_by_lines, word_a, word_b, word_c, topn)

    # Manejar errores si alguna palabra no está en el vocabulario
    if similar_by_pages is None or similar_by_lines is None:
        return

    # Convertir a listas formateadas
    words_pages = [f"{w[0]} ({w[1]:.2f})" for w in similar_by_pages]
    words_lines = [f"{w[0]} ({w[1]:.2f})" for w in similar_by_lines]

    # Asegurar que ambas listas tengan el mismo tamaño
    max_len = max(len(words_pages), len(words_lines))
    words_pages += [""] * (max_len - len(words_pages))
    words_lines += [""] * (max_len - len(words_lines))

    # Crear tabla formateada
    print("\n" + "=" * 60)
    print(f"Comparación para la analogía: '{word_a}' es a '{word_b}' lo que '{word_c}' es a:")
    print("=" * 60)
    print("   Model By Pages   |  Model By Lines ")
    print("-" * 60)

    for w1, w2 in zip(words_pages, words_lines):
        print(f" {w1.ljust(20)} | {w2}")

    print("-" * 60)  # Línea final separadora

# Lista de analogías a comparar
analogies = [
    ("princesa", "príncipe", "reina"),
    ("jeque", "tribu", "califa"),
    ("anillo", "engarce", "palacio"),
    ("sultán", "palacio", "mercader"),
]

# Generar comparación para cada analogía
for word_a, word_b, word_c in analogies:
    analogy_compare_models(w2v_model_by_pages, w2v_model_by_lines, word_a, word_b, word_c)


Comparación para la analogía: 'princesa' es a 'príncipe' lo que 'reina' es a:
   Model By Pages   |  Model By Lines 
------------------------------------------------------------
 hossein (0.32)       | púsose (0.30)
 rey (0.29)           | hossein (0.30)
 cargador (0.27)      | héroe (0.29)
 escuchamos (0.26)    | mostrándole (0.27)
 faruz (0.26)         | rey (0.26)
 derbas (0.26)        | nazarenos (0.26)
 rumzán (0.26)        | yamlika (0.26)
 pastelero (0.26)     | ¡durante (0.26)
 porteros (0.26)      | estrictamente (0.26)
 oriunda (0.26)       | cansaba (0.26)
------------------------------------------------------------

Comparación para la analogía: 'jeque' es a 'tribu' lo que 'califa' es a:
   Model By Pages   |  Model By Lines 
------------------------------------------------------------
 califal (0.28)       | castigarme (0.28)
 mí (0.27)            | continua (0.27)
 billah (0.26)        | avisarme (0.27)
 marchaban (0.25)     | devolverme (0.27)
 pida (0.25)          | ca

#### Análisis de Analogías

1. "princesa" es a "príncipe" lo que "reina" es a:

* El modelo basado en páginas tiene "rey" como una de las respuestas más cercanas, lo que es lógico dada la relación monárquica.
* El modelo basado en líneas sugiere "héroe" y "hossein", posiblemente reflejando asociaciones narrativas dentro del texto.
* Otras respuestas como "pastelero" o "porteros" en el modelo por páginas parecen menos relacionadas semánticamente con la analogía planteada.

2. "jeque" es a "tribu" lo que "califa" es a:

* En el modelo basado en páginas, "califal" aparece como la mejor opción, lo que tiene sentido, dado que "califa" está relacionado con "califato".
* El modelo basado en líneas genera respuestas menos relacionadas, como "castigarme" o "avisarme".
* La presencia de "emisarios" en el modelo por líneas podría estar relacionada con el contexto histórico de los califas y su diplomacia.

3. "anillo" es a "engarce" lo que "palacio" es a:

* El modelo basado en líneas sugiere "pórtico", "vestíbulo" y "harem", palabras asociadas a palacios.
* El modelo basado en páginas genera respuestas como "penetramos", "celda" y "gradas", lo que también está relacionado con estructuras arquitectónicas.
* La presencia de "astrolabio" en ambas listas sugiere que en la narrativa el término está asociado a palacios.

4. "sultán" es a "palacio" lo que "mercader es a:

* "tienda" aparece como la mejor respuesta en el modelo basado en páginas, lo que es una elección semánticamente coherente.
* En el modelo basado en líneas, aparecen palabras menos esperadas como "caballo" y "pasillo".
* "Khan" en el modelo basado en líneas podría referirse a posadas orientales, lo que tiene cierto grado de relación con mercaderes.

#### Conclusión

Los resultados muestran que el modelo basado en páginas tiende a producir respuestas más alineadas con el significado semántico esperado en cada analogía. En cambio, el modelo basado en líneas genera algunas respuestas coherentes, pero también incorpora términos menos esperados, lo que podría deberse a asociaciones específicas del contexto narrativo en el que aparecen las palabras. Esto sugiere que la segmentación del texto influye significativamente en las asociaciones semánticas que los modelos pueden capturar.